In [ ]:
import pandas as pd

from AssetAllocation.datamanger import datamanger as dm
from AssetAllocation.analytics import summary
from AssetAllocation.reporting import reports as rp, plots, formatter as fmt
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import stochMV as stMV
import seaborn as sns
from ipywidgets import interact, interact_manual

PLAN = 'IBT'

In [ ]:
#create plan object that has the relevant variables (cov, policy ret, var,fsv )
ts_dict = dm.get_ts_data(plan=PLAN)
ts_outputs = summary.get_ts_output(ts_dict)
# pp_inputs = summary.get_pp_inputs(mv_inputs, ts_dict)

# summarized_output = dm.get_plan_data('output.xlsx')
# plan = summary.get_plan_params(summarized_output)
plan = summary.get_plan_params(ts_outputs)

In [ ]:
#Display plan inputs
pp_dict = plan.get_pp_dict()
@interact
def display_pp_inputs(variable=pp_dict.keys()):
    if variable == 'Corr' or variable == 'Cov':
        return plots.draw_heatmap(pp_dict[variable], half=False)
    else:
        return fmt.get_plan_styler(pp_dict[variable])

In [ ]:
#initialize the stochastic mean variance and generate the random returns
s = stMV.stochMV(plan, 120)
s.generate_plans()

In [ ]:
#visualize the simulated returns
sns.pairplot(s.returns_df, corner=True)
plt.savefig("sampling.jpeg")

In [ ]:
#definition of the boundaries and constraints for the optimization
bnds = dm.get_bounds(plan=PLAN)
cons = ({'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.5},
        {'type': 'eq', 'fun': lambda x: np.sum(x[0:len(plan)-1]) - x[3] - .02})


In [ ]:
# computation of the efficient frontier for each of the simulated plan
# this takes about 5 min to run for 100 iterations, around an hour for 1000 iterations
s.generate_efficient_frontiers(bnds, cons,num_ports=25)

In [ ]:
# output the dataframe which has the average allocation
ports_df = s.opt_ports_df
fmt.get_port_styler(ports_df)

In [ ]:
#Asset Allocation Plot
aa_fig = plots.get_aa_fig(ports_df)
aa_fig.show()

In [ ]:
#Plotly version of the Efficient Frontier plot
ef_fig = plots.get_ef_fig(ports_df)
ef_fig.show()